In [1]:
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import scipy.io.wavfile
import tensorflow as tf
import tensorflow_io as tfio
from transfer import TimbreTransfer

Done!


In [9]:
# Load Data
df = px.data.tips()

# Build App

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.H1("Audio Style-Transfer Examples using DDSP"),
    dcc.RadioItems(id = 'song',
    options=[
        {'label': 'Happy Birthday', 'value': 'hbd.pkl'},
        {'label': 'Jolene', 'value': 'jolene-2.pkl'},
        {'label': 'Bad Bunny', 'value': 'feats.pkl'}
    ],
    value=None,labelStyle={'display': 'inline-block'}
)  ,
    dcc.RadioItems(id = 'instrument',
    options=[
        {'label': 'Clarinet', 'value': 'clarinet'},
        {'label': 'Flute', 'value': 'flute'},
        {'label': 'Trumpet', 'value': 'trumpet'},
        {'label': 'Violin', 'value': 'violin'}
    ],
    value=None,labelStyle={'display': 'inline-block'}
)  ,  html.Button('Transfer', id='submit-val', n_clicks=0),
    dcc.Loading(
            id="loading-1",
            type="default",
            children=html.Div(id='output-container')
        ),
])
# Define callback to update graph
@app.callback(
    Output("output-container", "children"),
    Input("submit-val", "n_clicks"),
    [State("song", "value"),State("instrument","value")],
)
def toggle_model(n1, song, instrument):
    if n1>0:
        model = TimbreTransfer(instrument,song)
        model.fit()
        out_audio = model.resynth()
        audio_noise = model.controls['filtered_noise']['signal']
        audio_harm = model.controls['harmonic']['signal']
        
        scipy.io.wavfile.write(f'assets/{song[:-4]}.wav', 16000, model.audio[0])
        scipy.io.wavfile.write(f'assets/{instrument}_{song[:-4]}.wav', 16000, out_audio.numpy()[0])
        scipy.io.wavfile.write(f'assets/{instrument}_{song[:-4]}_noise.wav', 16000, audio_noise.numpy()[0])
        scipy.io.wavfile.write(f'assets/{instrument}_{song[:-4]}_harmonic.wav', 16000, audio_harm.numpy()[0])
        
        orig_spec = tfio.experimental.audio.spectrogram(model.audio, nfft=512, window=512, stride=256)
        orig_spec = tfio.experimental.audio.melscale(orig_spec, rate=16000, mels=128, fmin=0, fmax=8000)
        orig_spec = tf.math.log(orig_spec).numpy()
        
        out_spec = tfio.experimental.audio.spectrogram(out_audio, nfft=512, window=512, stride=256)
        out_spec = tfio.experimental.audio.melscale(out_spec, rate=16000, mels=128, fmin=0, fmax=8000)
        out_spec = tf.math.log(out_spec).numpy()
        
        noise_spec = tfio.experimental.audio.spectrogram(audio_noise, nfft=512, window=512, stride=256)
        noise_spec = tfio.experimental.audio.melscale(noise_spec, rate=16000, mels=128, fmin=0, fmax=8000)
        noise_spec = tf.math.log(noise_spec).numpy()
        
        harm_spec = tfio.experimental.audio.spectrogram(audio_harm, nfft=512, window=512, stride=256)
        harm_spec = tfio.experimental.audio.melscale(harm_spec, rate=16000, mels=128, fmin=0, fmax=8000)
        harm_spec = tf.math.log(harm_spec).numpy()
        
        return html.Div([ 
            html.H4("Original:"),
            html.Audio(src=f'assets/{song[:-4]}.wav', controls=True),
            dcc.Graph(id="orig", figure = px.imshow(orig_spec[0].T), style={"display": "inline-block"}),
            html.H4(f"As {instrument}:"),
            html.Audio(src=f'assets/{instrument}_{song[:-4]}.wav', controls=True),
            dcc.Graph(id="out", figure = px.imshow(out_spec[0].T), style={"display": "inline-block"}),
            html.H4(f"Noise Component:"),
            html.Audio(src=f'assets/{instrument}_{song[:-4]}_noise.wav', controls=True),
            dcc.Graph(id="out", figure = px.imshow(noise_spec[0].T), style={"display": "inline-block"}),
            html.H4(f"Harmonic Component:"),
            html.Audio(src=f'assets/{instrument}_{song[:-4]}_harmonic.wav', controls=True),
            dcc.Graph(id="out", figure = px.imshow(harm_spec[0].T), style={"display": "inline-block"})
        ])
# Run app and display result inline in the notebook
app.run_server(mode='jupyterlab', debug=True)

Restoring clarinet
Restoring model took 3.5 seconds
